# Lab 10

In [1]:
!#git clone -b poleval/task3 https://github.com/n-waves/fastai --depth 1 

#### Imports

In [2]:
import os
from functional import seq
from IPython.core.display import HTML
from fastai.text import *

## Setup

In [3]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [4]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [5]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

### Coloring

In [6]:
from termcolor import colored


def highlighted(
        highlight,
        whole_text,
):
    return whole_text.replace(
        highlight, colored(highlight, color="green"))

## Sentencepiece

In [7]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)

spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")
# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 100)

True

In [8]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [9]:
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],fastai_model_path)
lm.reset()
lm.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [10]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

### From infer

In [20]:
def next_tokens(ids_, model, toks_at_once):  #toks_at_once wont be used
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    model.reset()  # todo:bcm - do or dont'?
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]
    
    #     arg = torch.sum(p[:-2],0)
    arg =  p[-1]
#     arg, _  = torch.max(p,dim=0)
#     print(arg.size())
    r = int(torch.argmax(arg))
#     r = int(torch.multinomial(p[-1].exp(), 1))


    while r in [ids_[-1],24]: #, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids


def next_word(ss, model, toks_at_once):
    ids = spm_processor.encode_as_ids(ss)
    return spm_processor.decode_ids(next_tokens(ids, model, toks_at_once))

In [21]:
def next_words_good(ss, lm, n_words, finishers=set([UNK_ID])):
    initial_wip = spm_processor.encode_as_ids(ss)
    wip = initial_wip
    for i in range(n_words):
        wip = wip + next_tokens(wip, lm, 1)

    print(seq(wip).drop(len(initial_wip)))
    return spm_processor.decode_ids(wip)
#         seq(wip).take_while(lambda x: x not in finishers).to_list())

In [22]:
def next_words_bad(ss, lm, n_words, toks_at_once=1):
    wip = ss
    for i in range(n_words):
        wip = wip + " " + next_word(wip, lm, toks_at_once)
    return wip

In [23]:
lm.reset()

In [24]:
spm_processor.decode_ids([7])

''

In [25]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

In [26]:
for sentence in sentences:
    print("")
    lm.reset()
    print(highlighted(sentence,next_words_good(sentence, lm, 30)))
    lm.reset()
    print(highlighted(sentence,next_words_bad(sentence, lm, 30)))


[1045, 2, 61, 73, 369, 41397, 4, 11274, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7]
Warszawa to największe miasto Po Nanadtowroc ,kiej  , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇  
Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce . . . . . . . . . . . . . . . . . . . .

[558, 2, 61, 73, 369, 41397, 4, 11274, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7]
Te zabawki należą do najbardziej Po Nanadtowroc ,kiej  , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇  
Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce . . . . . . . . . . . . . . . . . . . . . . .

[4, 2, 61, 73, 369, 41397, 4, 11274, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7]
Policjant przygląda się , Po Nanadtowroc ,kiej  , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇  
Policjant przygląda się , jak a ntyterrorystyczn y                         

[2973, 2, 61, 73, 369, 41397, 4, 11274, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7,

## Playground

In [18]:
inpucik = "Mama bardzo dobrze wydała swoje ulubione "
test_ids = [np.array(spm_processor.encode_as_ids(inpucik))]
test_ids

[array([    2,   618,     8,    91,   431, 11111,   344, 30517,     3])]

In [19]:
test_ds = LMTextDataset(test_ids)
test_samp = SortSampler(test_ids, key=lambda x: len(test_ids[x]))
test_dl = DataLoader(test_ds,
                     bs,
                     transpose=True,
                     transpose_y=True,
                     num_workers=1,
                     pad_idx=PAD_ID,
                     sampler=test_samp,
                     pre_pad=False)
x, y = list(test_dl)[0]
x2 = [int(i) for i in x]
# y2 = [int(i) for i in y]
spm_processor.decode_ids(x2)

'Mama bardzo dobrze wydała swoje ulubione'

In [243]:
def getp():
    dl = test_dl
    loss = 0.0
    tensor1 = None
    lm.reset()
    with no_grad_context():
        for (x, y) in tqdm(dl):
            targets = y.view(-1)
            preds = lm(x)[0]
            tensor1 = lm(x)
            not_pads = targets != PAD_ID
            ce = F.cross_entropy(preds[not_pads],
                                 targets[not_pads],
                                 reduction='sum')
            loss += ce
    loss
    p = tensor1[0]
    print(p[0].size())
    print([type(i) for i in tensor1])
    return p

In [244]:
def get_prediction(n):
    p = getp()
    print(p.size())

    #     for pi in p:
    #         r = torch.multinomial(pi.exp(), n)
    #         predicted_ids = [int(r_) for r_ in r]
    #         print(spm_processor.decode_ids(predicted_ids))
    #         print("")
    #         print("")

    r = torch.multinomial(p[-1].exp(), 10)
    print(r)
    print([p[-1][i] for i in r])
    predicted_ids = [int(r_) for r_ in r]
    return spm_processor.decode_ids(predicted_ids)

In [245]:
inpucik

'Mama bardzo dobrze wydała swoje ulubione '

In [246]:
p = getp()

100%|██████████| 1/1 [00:00<00:00, 14.31it/s]
torch.Size([50000])
[<class 'torch.Tensor'>, <class 'list'>, <class 'list'>]


In [247]:
from functional import seq

In [248]:
seq(i for i in
    p[-1]).map(lambda x: float(x)).enumerate().max_by(lambda id_val: id_val[1])

[4838, 4.52532958984375]

In [250]:
spm_processor.decode_ids([4838])

'piosenki'

In [215]:
get_prediction(20)
np.array(p[-1])

100%|██████████| 1/1 [00:00<00:00, 16.97it/s]
torch.Size([50000])
[<class 'torch.Tensor'>, <class 'list'>, <class 'list'>]
torch.Size([7, 50000])
tensor([48758, 47070, 15515, 38977, 49387, 24972,  8005,  3566, 10795, 17061],
       device='cuda:0')
[tensor(-0.4503, device='cuda:0'), tensor(-0.5835, device='cuda:0'), tensor(-0.0726, device='cuda:0'), tensor(-2.5826, device='cuda:0'), tensor(-0.7917, device='cuda:0'), tensor(2.1340, device='cuda:0'), tensor(1.7828, device='cuda:0'), tensor(-4.4941, device='cuda:0'), tensor(-0.9323, device='cuda:0'), tensor(-0.6776, device='cuda:0')]


TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.